In [51]:
import sys
sys.path.append('..')
import os
import json
import glob
import shutil
import pandas as pd
import numpy as np
import open3d as od
from utils.cow import get_cow_regno

In [52]:
def get_from_nias_ground_truth_file(cow_regno, ground_truth_dataframe, score_column='bcs'):
    df = ground_truth_dataframe[ground_truth_dataframe['cow_regno']==cow_regno]
    if len(df) != 1:
        raise ValueError("Problem with groundtruth of this cow_regno.")
    score = df[[score_column]].values[0][0]
    return score

In [53]:
with open('./config.json', 'r') as fp:
    config = json.load(fp)

In [54]:
pcd_paths = config['pcd_paths']
json_paths = config['json_paths']
rump_path = config['rump_path']
label_score_path = config['label_score_path']
if os.path.exists(rump_path):
    shutil.rmtree(rump_path)
os.mkdir(rump_path)

In [55]:
ground_truth_dataframe1 = pd.read_excel("E:/ZOOTOS RnD/NIAS/Data/DataTest/ground_truth.xlsx")
ground_truth_dataframe2 = pd.read_excel("E:/ZOOTOS RnD/NIAS/Data/DataTest/ground_truth2.xlsx")

In [56]:
print('bcs' in ground_truth_dataframe1.columns)
print('bcs' in ground_truth_dataframe2.columns)

True
True


In [57]:
bname = 'Rump'

In [58]:
cows, bcs_score = [], []
center_x, center_y, center_z = [], [], []
count = 0
nolabel = []
for json_path, pcd_path in zip(json_paths, pcd_paths):
    list_json = glob.glob(f'{json_path}/*.json')
    list_json = [fname for fname in list_json if not fname.endswith('classes.json')]
    for json_path in list_json:
        cow_regno = get_cow_regno(json_path)
        try:
            score = get_from_nias_ground_truth_file(cow_regno, ground_truth_dataframe1, 'bcs')
        except:
            try:
                score = get_from_nias_ground_truth_file(cow_regno, ground_truth_dataframe2, 'bcs')
            except Exception as e:
                print(e, cow_regno, json_path)
                nolabel.append(cow_regno)
                continue
        with open(json_path, 'r') as f:
            box_data = json.load(f)

        try:
            if len(box_data['objects']) == 0:
                print("Not labeled yet.")
                continue
        except:
            print("Not labeled yet")
            continue
        bboxes = {obj['name']: np.array(obj['vertices']) for obj in box_data['objects']}
        if bname not in bboxes.keys():
            print("Name error")
            continue

        pcd = od.io.read_point_cloud(f"{pcd_path}/{cow_regno}.ply")
        points = np.asarray(pcd.points)

        bbox = bboxes['Rump']
        
        p0 = bbox[0]
        p1 = bbox[3]
        p2 = bbox[1]
        p3 = bbox[4]

        # p0 as origin
        v1 = p1 - p0
        v2 = p2 - p0
        v3 = p3 - p0

        u = points - p0
        u_dot_v1 = u.dot(v1)
        u_dot_v2 = u.dot(v2)
        u_dot_v3 = u.dot(v3)

        inside_v1 = np.logical_and(np.sum(v1**2) > u_dot_v1, u_dot_v1 > 0)
        inside_v2 = np.logical_and(np.sum(v2**2) > u_dot_v2, u_dot_v2 > 0)
        inside_v3 = np.logical_and(np.sum(v3**2) > u_dot_v3, u_dot_v3 > 0)

        points_inside = np.logical_and(np.logical_and(inside_v1, inside_v2), inside_v3)
        idx = np.where(points_inside == True)[0]
        part = pcd.select_by_index(idx)
        center = np.asarray(part.points).mean(axis=0)
        part = part.translate(-center)
        x, y, z = np.asarray(part.points).mean(axis=0)
        center_x.append(x)
        center_y.append(y)
        center_z.append(z)
        od.io.write_point_cloud(f"{rump_path}/{cow_regno}.ply", part)
        cows.append(cow_regno)
        bcs_score.append(score)
        print(f"Done: {cow_regno}")
        count += 1
print('Total rumps: ', count)

Done: 500991129
Done: 501034812
Done: 501049585
Done: 501049591
Done: 501051848
Done: 501063723
Done: 501066745
Done: 501069296
Done: 501087052
Done: 501093021
Done: 501097647
Done: 501112340
Done: 501125582
Done: 501129559
Done: 501146514
Done: 501148583
Done: 501165161
Done: 501165422
Done: 501170595
Done: 501175973
Not labeled yet.
Done: 501177573
Done: 501177584
Done: 501181588
Done: 501188465
Done: 501189051
Done: 501196133
Done: 501196647
Done: 501201401
Done: 501207139
Done: 501209155
Done: 501210889
Done: 501212459
Done: 501213403
Done: 501215767
Done: 501217300
Not labeled yet.
Done: 501220840
Not labeled yet.
Done: 501226073
Done: 501228692
Done: 501230562
Done: 501236151
Done: 501236190
Done: 501236191
Done: 501236224
Done: 501236226
Not labeled yet.
Done: 501237218
Done: 501238312
Done: 501239084
Not labeled yet.
Done: 501239785
Done: 501240329
Done: 501242702
Done: 501250482
Done: 501253980
Done: 501253985
Done: 501255892
Done: 501258818
Done: 501258954
Done: 501262741
Don

In [48]:
print(len(cows), len(bcs_score))

403 403


In [49]:
label = pd.DataFrame({'cow_regno': cows, 'bcs': bcs_score, 'center_x': center_x, 'center_y': center_y, 'center_z': center_z})
label.to_csv(label_score_path, index=False)

In [50]:
len(nolabel)

5